In [12]:
import sys
sys.path.insert(0, '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/US/')

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import pickle
import random
from scipy import stats
import plotly.express as px
from sklearn.metrics import mean_squared_error, r2_score

import os
from pipeline_config import *


In [14]:
#Seeds for reproducability:

torch.manual_seed(1)
np.random.seed(2)
random.seed(3)

In [15]:
with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/US/bidirectional/test_predictions.pickle', 'rb') as f:
    prediction_dic = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/US/bidirectional/test_predictions_1.pickle', 'rb') as f:
    prediction_dic_no_1 = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/US/bidirectional/test_predictions_2.pickle', 'rb') as f:
    prediction_dic_no_2 = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/US/bidirectional/test_predictions_3.pickle', 'rb') as f:
    prediction_dic_no_3 = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/pickle_files/bi_directional_us_dataset_dict.pickle', 'rb') as f:
    raw_dataset_dict = pickle.load(f)
    
with open(category_id_to_category_name_path, 'rb') as f:
    category_id_to_name_dict = pickle.load(f)
    
with open(categories_per_indent_path, 'rb') as f:
    categories_per_indent_dict = pickle.load(f)

In [16]:
def create_test_dataframe(raw_dataset_dict: dict):
    test_dict = {}
    for key in raw_dataset_dict.keys():
    #for key in ['All items']:
        df = raw_dataset_dict[key][['Category', 'Date', 'Year', 'Indent', 'Inflation t+1']]
        df.dropna(inplace=True)
        df.rename(columns={'Inflation t+1': 'Actual'}, inplace=True)
        target_df = df[df['Year'] > Year]
        test_dict[key] = target_df
    return test_dict

test_dict = create_test_dataframe(raw_dataset_dict)


/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_70070/1267119108.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_70070/1267119108.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Inflation t+1': 'Actual'}, inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_70070/1267119108.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

In [17]:
def get_df_with_predictions(prediction_dic, prediction_dic_1: dict, prediction_dic_2: dict, prediction_dic_3: dict, dict_of_categories_df:dict) -> dict:
    all_data_dict = {}
    for key in list(prediction_dic.keys()):
        predictions = prediction_dic[key]
        predictions_1 = prediction_dic_1[key]
        predictions_2 = prediction_dic_2[key]
        predictions_3 = prediction_dic_3[key]

        prediction_df =  pd.DataFrame(predictions.flatten().detach().numpy())
        prediction_df.rename(columns = {0: 'Prediction'}, inplace=True)
        prediction_1_df =  pd.DataFrame(predictions_1.flatten().detach().numpy())
        prediction_1_df.rename(columns = {0: 'Prediction_No_1'}, inplace=True)
        prediction_2_df =  pd.DataFrame(predictions_2.flatten().detach().numpy())
        prediction_2_df.rename(columns = {0: 'Prediction_No_2'}, inplace=True)
        prediction_3_df =  pd.DataFrame(predictions_3.flatten().detach().numpy())
        prediction_3_df.rename(columns = {0: 'Prediction_No_3'}, inplace=True)
        dict_of_categories_df[key] = dict_of_categories_df[key].reset_index(drop=True)

        all_data_dict[key] = pd.concat([dict_of_categories_df[key], prediction_df, prediction_1_df, prediction_2_df, prediction_3_df], axis=1)
    return all_data_dict

all_data_test_dict=get_df_with_predictions(prediction_dic, prediction_dic_no_1, prediction_dic_no_2, prediction_dic_no_3, test_dict)

In [18]:
# sanity check
all_categories = list(all_data_test_dict.keys())
len(all_categories)

349

In [25]:
def avg_rmse(all_data_test_dict):
    mse_lst = []
    mse_lst_1 = []
    mse_lst_2 = []
    mse_lst_3 = []
    for key in all_categories:
        df_predictions = all_data_test_dict[key]
        y_actual = df_predictions['Actual'].values

        y_pred = df_predictions['Prediction'].values
        mse = mean_squared_error(y_pred, y_actual)
        mse_lst.append(mse)

        y_pred_1 = df_predictions['Prediction_No_1'].values
        mse_1 = mean_squared_error(y_pred_1, y_actual)
        mse_lst_1.append(mse_1)

        y_pred_2 = df_predictions['Prediction_No_2'].values
        mse_2 = mean_squared_error(y_pred_2, y_actual)
        mse_lst_2.append(mse_2)

        y_pred_3 = df_predictions['Prediction_No_3'].values
        mse_3 = mean_squared_error(y_pred_3, y_actual)
        mse_lst_3.append(mse_3)

    rmse_list = list(map(np.sqrt,mse_lst))
    avg_rmse = np.mean(rmse_list)
    rmse_std = np.std(rmse_list)
    
    rmse_list_1 = list(map(np.sqrt,mse_lst_1))
    avg_rmse_1 = np.mean(rmse_list_1)
    rmse_std_1 = np.std(rmse_list_1)

    rmse_list_2 = list(map(np.sqrt,mse_lst_2))
    avg_rmse_2 = np.mean(rmse_list_2)
    rmse_std_2 = np.std(rmse_list_2)

    rmse_list_3 = list(map(np.sqrt,mse_lst_3))
    avg_rmse_3 = np.mean(rmse_list_3)
    rmse_std_3 = np.std(rmse_list_3)
    
    print(f'RMSE:  {avg_rmse}')
    print(f'RMSE - No Loss Term 1:  {avg_rmse_1}, difference = {avg_rmse_1-avg_rmse}')
    print(f'RMSE - No Loss Term 2:  {avg_rmse_2}, difference = {avg_rmse_2-avg_rmse}')
    print(f'RMSE - No Loss Term 3:  {avg_rmse_3}, difference = {avg_rmse_3-avg_rmse}')
    print('-----------------------------------------------------------------------------')
    print(f'MSE std:  {rmse_std}')
    print(f'MSE std - No Loss Term 1:  {rmse_std_1}')
    print(f'MSE std - No Loss Term 2:  {rmse_std_2}')
    print(f'MSE std - No Loss Term 3:  {rmse_std_3}')

    return avg_rmse, rmse_std, avg_rmse_1,rmse_std_1, avg_rmse_2,rmse_std_2, avg_rmse_3,rmse_std_3

avg_rmse(all_data_test_dict)

RMSE:  1.3342502971109562
RMSE - No Loss Term 1:  1.3462471866471377, difference = 0.011996889536181499
RMSE - No Loss Term 2:  1.3409948508301528, difference = 0.006744553719196578
RMSE - No Loss Term 3:  1.342833568650666, difference = 0.008583271539709836
-----------------------------------------------------------------------------
MSE std:  1.0723568602393183
MSE std - No Loss Term 1:  1.0895809256272573
MSE std - No Loss Term 2:  1.0856017696880786
MSE std - No Loss Term 3:  1.0849404546834835


(1.3342502971109562,
 1.0723568602393183,
 1.3462471866471377,
 1.0895809256272573,
 1.3409948508301528,
 1.0856017696880786,
 1.342833568650666,
 1.0849404546834835)

In [20]:
df_predictions = all_data_test_dict['All items']
y_actual = df_predictions['Actual'].values

y_pred = df_predictions['Prediction'].values
mse = mean_squared_error(y_pred, y_actual)
rmse = np.sqrt(mse)
print(f'Headline RMSE: {rmse}')

y_pred_1 = df_predictions['Prediction_No_1'].values
mse_1 = mean_squared_error(y_pred_1, y_actual)
rmse_1 = np.sqrt(mse_1)
print(f'Headline RMSE - No Loss Term 1: {rmse_1}, difference = {rmse_1-rmse}')

y_pred_2 = df_predictions['Prediction_No_2'].values
mse_2 = mean_squared_error(y_pred_2, y_actual)
rmse_2 = np.sqrt(mse_2)
print(f'Headline RMSE - No Loss Term 2: {rmse_2}, difference = {rmse_2-rmse}')

y_pred_3 = df_predictions['Prediction_No_3'].values
mse_3 = mean_squared_error(y_pred_3, y_actual)
rmse_3 = np.sqrt(mse_3)
print(f'Headline RMSE - No Loss Term 3: {rmse_3}, difference = {rmse_3-rmse}')

Headline RMSE: 0.31467639316317153
Headline RMSE - No Loss Term 1: 0.3347639970161361, difference = 0.020087603852964542
Headline RMSE - No Loss Term 2: 0.35451156809040824, difference = 0.039835174927236705
Headline RMSE - No Loss Term 3: 0.3547852597185258, difference = 0.04010886655535428


In [24]:
def total_corr(all_data_test_dict):
    corr_dict = {}
    corr_dict_1 = {}
    corr_dict_2 = {}
    corr_dict_3 = {}

    for key in all_categories:
        df_predictions = all_data_test_dict[key]
        y_actual = df_predictions['Actual'].values

        y_pred = df_predictions['Prediction'].values
        corr = stats.pearsonr(y_pred,y_actual)[0]
        corr_dict[key] =  corr

        y_pred_1 = df_predictions['Prediction_No_1'].values
        corr_1 = stats.pearsonr(y_pred_1,y_actual)[0]
        corr_dict_1[key] =  corr_1

        y_pred_2 = df_predictions['Prediction_No_2'].values
        corr_2 = stats.pearsonr(y_pred_2,y_actual)[0]
        corr_dict_2[key] =  corr_2

        y_pred_3 = df_predictions['Prediction_No_3'].values
        corr_3 = stats.pearsonr(y_pred_3,y_actual)[0]
        corr_dict_3[key] =  corr_3

    total_corr = sum(corr_dict.values())
    total_corr_1 = sum(corr_dict_1.values())
    total_corr_2 = sum(corr_dict_2.values())
    total_corr_3 = sum(corr_dict_3.values())
    
    print(f'total_corr = {total_corr}')
    print(f'total corr - No Loss Term 1: {total_corr_1}, difference = {total_corr-total_corr_1}')
    print(f'total corr - No Loss Term 2: {total_corr_2}, difference = {total_corr-total_corr_2}')
    print(f'total corr - No Loss Term 3: {total_corr_3}, difference = {total_corr-total_corr_3}')
    
    return total_corr, total_corr_1, total_corr_2, total_corr_3

total_corr(all_data_test_dict)

total_corr = 82.57005916566945
total corr - No Loss Term 1: 78.74287620901111, difference = 3.8271829566583335
total corr - No Loss Term 2: 78.54408416440107, difference = 4.025975001268378
total corr - No Loss Term 3: 76.679216558808, difference = 5.8908426068614546


(82.57005916566945, 78.74287620901111, 78.54408416440107, 76.679216558808)

In [23]:
def total_corr_all_items(all_data_test_dict):
    corr_dict = {}
    corr_dict_1 = {}
    corr_dict_2 = {}
    corr_dict_3 = {}

    for key in ['All items']:
        df_predictions = all_data_test_dict[key]
        y_actual = df_predictions['Actual'].values

        y_pred = df_predictions['Prediction'].values
        corr = stats.pearsonr(y_pred,y_actual)[0]
        corr_dict[key] =  corr

        y_pred_1 = df_predictions['Prediction_No_1'].values
        corr_1 = stats.pearsonr(y_pred_1,y_actual)[0]
        corr_dict_1[key] =  corr_1

        y_pred_2 = df_predictions['Prediction_No_2'].values
        corr_2 = stats.pearsonr(y_pred_2,y_actual)[0]
        corr_dict_2[key] =  corr_2

        y_pred_3 = df_predictions['Prediction_No_3'].values
        corr_3 = stats.pearsonr(y_pred_3,y_actual)[0]
        corr_dict_3[key] =  corr_3

    total_corr = sum(corr_dict.values())
    total_corr_1 = sum(corr_dict_1.values())
    total_corr_2 = sum(corr_dict_2.values())
    total_corr_3 = sum(corr_dict_3.values())
    
    print(f'Headline Corr = {total_corr}')
    print(f'Headline corr - No Loss Term 1: {total_corr_1}, difference = {total_corr-total_corr_1}')
    print(f'Headline corr - No Loss Term 2: {total_corr_2}, difference = {total_corr-total_corr_2}')
    print(f'Headline corr - No Loss Term 3: {total_corr_3}, difference = {total_corr-total_corr_3}')
    
    return total_corr, total_corr_1, total_corr_2, total_corr_3

total_corr_all_items(all_data_test_dict)

Headline Corr = 0.46711392575654265
Headline corr - No Loss Term 1: 0.4297082195547561, difference = 0.03740570620178657
Headline corr - No Loss Term 2: 0.283928110957805, difference = 0.18318581479873763
Headline corr - No Loss Term 3: 0.3278388139238024, difference = 0.13927511183274027


(0.46711392575654265,
 0.4297082195547561,
 0.283928110957805,
 0.3278388139238024)